In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
pip install underthesea

     |████████████████████████████████| 7.6 MB 5.3 MB/s 
     |████████████████████████████████| 965 kB 44.2 MB/s 
     |████████████████████████████████| 235 kB 47.3 MB/s 
     |████████████████████████████████| 581 kB 53.5 MB/s 


In [183]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from underthesea import sent_tokenize,word_tokenize
from itertools import chain
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import *
from sklearn.utils.class_weight import compute_class_weight
import string
import re
from os.path import join

In [5]:
root_data="/content/drive/MyDrive/vietnamese_students_feedback/preproccesd_data"

In [6]:
train=pd.read_csv(f'{root_data}/train.csv')

In [7]:
train

,sentence,sentiment,topic,cleaned_sentence,seq_len
0,giọng cô nghe dễ ngủ !,1,0,giọng cô nghe dễ ngủ,5
1,phòng thực hành nhỏ không đủ máy cho sinh viên...,0,2,phòng thực hành nhỏ không đủ máy cho sinh viên...,14
2,nội dung học cần cải thiện .,0,1,nội dung học cần cải thiện,6
3,làm nhiều bài kiểm tra quá .,0,1,làm nhiều bài kiểm tra quá,6
4,phần thi nói giữa các lớp không thống nhất về ...,0,1,phần thi nói giữa các lớp không thống nhất về ...,15
...,...,...,...,...,...
11421,cô đưa ra nhiều hướng giải quyết khi sinh viên...,2,0,cô đưa ra nhiều hướng giải quyết khi sinh viên...,16
11422,em nghĩ nội dung thực hành nên gần với đồ án h...,0,1,em nghĩ nội dung thực hành nên gần với đồ án hơn,12
11423,luôn nhiệt tình và tận tâm .,2,0,luôn nhiệt tình và tận tâm,6
11424,nó rất rập khuôn và không thể khiến cho sinh v...,0,1,nó rất rập khuôn và không thể khiến cho sinh v...,15


In [23]:
valid=pd.read_csv(f'{root_data}/valid.csv')

In [17]:
def tokenizer(s): 
    sentences = sent_tokenize(s) 
    # tokens=[]
    _sentences=[]
    for sentence in sentences:
      # tokens+=sentence
      tokens=[]
      for token in word_tokenize(sentence,):
        token=token.replace("-", " ")
        token=token.replace(" ","_")
        
        tokens.append(token)
      _sentences.append(tokens)
    return list(chain.from_iterable(_sentences))

In [18]:
tokenizer("làm nhiều bài kiểm tra quá .")

['làm', 'nhiều', 'bài', 'kiểm_tra', 'quá', '.']

In [19]:
train_x=train['cleaned_sentence'].tolist()

In [24]:
valid_x=valid['cleaned_sentence'].tolist()

In [20]:
train_y=train['topic'].tolist()

In [25]:
valid_y=valid['topic'].tolist()

#Convert text to count vectorizer

In [124]:
count_vectorizer=CountVectorizer(tokenizer=tokenizer,ngram_range=(1,3),max_features=5000)

In [125]:
train_vectorizer=count_vectorizer.fit_transform(train_x)

In [132]:
vocabs=count_vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [135]:
len(vocabs)

5000

In [136]:
vocab2idx={vocab:idx for idx,vocab in enumerate(vocabs)}

In [126]:
# class_weight=compute_class_weight(y=train_y,classes=[0,1,2,3],class_weight='balanced')
# class_weight

# K-fold SVM

In [127]:
svm_clf=SVC(class_weight='balanced')

In [128]:
param_grid={'C':[1e-5,1e-3,1e-1,1,10,100]}
svm_cv=GridSearchCV(estimator=svm_clf,param_grid=param_grid,cv=5,scoring='f1_weighted')

In [129]:
svm_cv.fit(train_vectorizer,train_y)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced'),
             param_grid={'C': [1e-05, 0.001, 0.1, 1, 10, 100]},
             scoring='f1_weighted')

In [130]:
svm_cv.best_params_

{'C': 10}

In [131]:
svm_cv.best_score_

0.8355534601177199

In [158]:
def create_inputs(sample):
  tokens=tokenizer(sample)
  size_samples=len(tokens)
  vectors=np.zeros((1,len(vocabs)))
  
  #unigram
  for token in tokens:
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  #bigram
  for idx in range(size_samples-1):
    token=" ".join(tokens[idx:idx+2])
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  #trigram
  for idx in range(size_samples-2):
    token=" ".join(tokens[idx:idx+3])
    if token in vocab2idx:
      vectors[:,vocab2idx[token]]+=1

  return vectors



#Classifier on valid set

In [163]:
valid_vectorizer=[create_inputs(sample) for sample in valid_x]

In [164]:
valid_vectorizer=np.concatenate(valid_vectorizer,axis=0)

In [171]:
valid_pred=svm_cv.predict(valid_vectorizer).tolist()

In [173]:
confusion_matrix(valid_pred,valid_y)

array([[1054,   58,    8,   31],
       [  53,  194,   14,   17],
       [   4,    4,   45,    2],
       [  40,   11,    3,   45]])

In [174]:
print(classification_report(valid_pred,valid_y,digits=5))

              precision    recall  f1-score   support

           0    0.91573   0.91573   0.91573      1151
           1    0.72659   0.69784   0.71193       278
           2    0.64286   0.81818   0.72000        55
           3    0.47368   0.45455   0.46392        99

    accuracy                        0.84523      1583
   macro avg    0.68971   0.72157   0.70289      1583
weighted avg    0.84538   0.84523   0.84488      1583



#Classifier on test set

In [176]:
test=pd.read_csv('/content/drive/MyDrive/vietnamese_students_feedback/raw_dataset/test.csv')

In [178]:
punctuations=list(string.punctuation)

In [181]:
def normalizer(text,joinf=False):
  def remove_symbols(s):
    return re.sub(r'\b(colon\w+)\b'," ",s)

  def remove_fraction(s):
    return re.sub(r'\b(fraction)\b','/',s)

  def replace_dot(s):
    return s.replace('dot','.')
    # return re.sub(r'\b(dot)\b',".",s)

  def replace_doubledot(s):
    return s.replace('doubledot',':')
    
  def uncased(s):
    return s.lower()

  def fix_whitespaces(s):
    return re.sub(r'\s+',' ',s)

  def remove_entities(s):
    return re.sub(r'\b((\w+|)wzjwz\d+)\b'," ",s)

  def remove_vv(s):
    return s.replace('v.v'," ")

  def remove_punctuations(s):
      return ''.join(ch for ch in s if ch not in punctuations)
  def norm_major(s):
      majors={
          'cnpm':"công nghệ phần mềm",
          'khmt':"khoa học máy tính",
          'ktmt':"kĩ thuật máy tính",
          'cntt':"công nghệ thông tin",
          'av1':"anh văn 1",
          'av2':"anh văn 2",
          'av3':"anh văn 3"
      }
      for k,v in majors.items():
        s=re.sub(r'\b({})\b'.format(k),v,s)
      return s
  def remove_ps(s):
    return s.replace("p / s"," ")

  def tokenize(s,join=False): 
    sentences = sent_tokenize(s) 
    # tokens=[]
    _sentences=[]
    for sentence in sentences:
      # tokens+=sentence
      tokens=[]
      for token in word_tokenize(sentence,):
        token=token.replace("-", " ")
        if join==True:
          token=token.replace(" ","_")
        
        tokens.append(token)
      _sentences.append(" ".join(tokens))
    return " ".join(_sentences)
  
  return fix_whitespaces(uncased(remove_punctuations(tokenize(norm_major(remove_ps(remove_fraction(remove_vv(remove_entities(remove_symbols(replace_dot(replace_doubledot(text)))))))),join)))).strip()

In [186]:
test_x=test['sentence'].map(normalizer).tolist()

In [187]:
test_y=test['topic'].tolist()

In [188]:
test_vectorizer=[create_inputs(sample) for sample in test_x]

In [189]:
test_vectorizer=np.concatenate(test_vectorizer,axis=0)

In [190]:
test_pred=svm_cv.predict(test_vectorizer).tolist()

In [191]:
confusion_matrix(test_pred,test_y)

array([[2090,  150,   15,   36],
       [ 130,  388,   17,   39],
       [   3,    4,  109,    3],
       [  67,   30,    4,   81]])

In [192]:
print(classification_report(test_pred,test_y,digits=5))

              precision    recall  f1-score   support

           0    0.91266   0.91227   0.91246      2291
           1    0.67832   0.67596   0.67714       574
           2    0.75172   0.91597   0.82576       119
           3    0.50943   0.44505   0.47507       182

    accuracy                        0.84270      3166
   macro avg    0.71304   0.73731   0.72261      3166
weighted avg    0.84095   0.84270   0.84140      3166

